In [5]:
import logging
import torch
from transformers import pipeline
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("salesken/query_wellformedness_score") 
model = AutoModelForSequenceClassification.from_pretrained("salesken/query_wellformedness_score")# model for well formedness score

Some weights of the model checkpoint at salesken/query_wellformedness_score were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [6]:
df=pd.read_csv('..\\top_results.csv', index_col=0)

In [48]:
scores=[]
# get a list of scores
for i in range(0,len(df)):
    features = tokenizer(df.at[i,"text"],  padding=True, truncation=True, return_tensors="pt")
    model.eval()
    with torch.no_grad():
        scores.append(model(**features).logits)
    if i%1000==0:
        logging.info("Rows done= {a} , {c} % done ".format(a=i,c=100*i/(len(df))))

INFO:root:Rows done= 0 , 0.0 % done 
INFO:root:Rows done= 1000 , 0.6707672907038361 % done 
INFO:root:Rows done= 2000 , 1.3415345814076722 % done 
INFO:root:Rows done= 3000 , 2.0123018721115082 % done 
INFO:root:Rows done= 4000 , 2.6830691628153445 % done 
INFO:root:Rows done= 5000 , 3.3538364535191807 % done 
INFO:root:Rows done= 6000 , 4.0246037442230165 % done 
INFO:root:Rows done= 7000 , 4.695371034926853 % done 
INFO:root:Rows done= 8000 , 5.366138325630689 % done 
INFO:root:Rows done= 9000 , 6.036905616334525 % done 
INFO:root:Rows done= 10000 , 6.707672907038361 % done 
INFO:root:Rows done= 11000 , 7.378440197742197 % done 
INFO:root:Rows done= 12000 , 8.049207488446033 % done 
INFO:root:Rows done= 13000 , 8.719974779149869 % done 
INFO:root:Rows done= 14000 , 9.390742069853706 % done 
INFO:root:Rows done= 15000 , 10.061509360557542 % done 
INFO:root:Rows done= 16000 , 10.732276651261378 % done 
INFO:root:Rows done= 17000 , 11.403043941965214 % done 
INFO:root:Rows done= 18000 ,

In [73]:
result = [tensor.item() for tensor in scores] #convert tensor to float
df["Formness_score"]=result 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149083 entries, 0 to 149082
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   text            149083 non-null  object 
 1   Company         149083 non-null  object 
 2   year            149083 non-null  int64  
 3   Label           149083 non-null  object 
 4   sim_score       149083 non-null  float64
 5   Formness_score  149083 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 12.0+ MB


In [52]:
df.to_csv("formness_data.csv")